# Transforming Data

In [140]:
# Import Raw Data
import pandas as pd
import os
import yfinance as yf
scrapped_data = pd.read_csv('https://raw.githubusercontent.com/fauzanrrizky/mini-projects/main/IDX%20Financial%20Analysis/datasets/lq45_financial_data_raw.csv')
scrapped_data

,symbol,account,type,2019,2020,2021,2022,2023
0,ACES,Accounts Payable,BS,NaN,1.642272e+11,1.267238e+11,1.443501e+11,1.256231e+11
1,ACES,Accounts Receivable,BS,NaN,1.434827e+11,6.677764e+10,8.585493e+10,1.986098e+11
2,ACES,Accumulated Depreciation,BS,NaN,-8.396180e+11,-1.867399e+12,-1.921765e+12,-2.003892e+12
3,ACES,Additional Paid In Capital,BS,NaN,4.405749e+11,4.405749e+11,4.405749e+11,4.405749e+11
4,ACES,Allowance For Doubtful Accounts Receivable,BS,NaN,-5.511007e+07,-2.593056e+08,-3.957857e+06,-1.799964e+07
...,...,...,...,...,...,...,...,...
7186,UNVR,Total Unusual Items,IS,-2.479000e+09,NaN,NaN,NaN,NaN
7187,UNVR,Total Unusual Items Excluding Goodwill,IS,-2.479000e+09,NaN,NaN,NaN,NaN
7188,UNVR,Treasury Shares Number,BS,NaN,NaN,NaN,NaN,0.000000e+00
7189,UNVR,Work In Process,BS,NaN,9.579300e+10,6.408100e+10,6.155200e+10,8.237200e+10


## Drop unused rows and columns

In [112]:
scrapped_data = scrapped_data.drop(columns=['2019'])

### Choose relevant rows

Drop all rows from df account, except:
- Total Revenue
- Net Income
- Total Assets
- Total Liabilities Net Minority Interest
- Cash Flowsfromusedin Operating Activities Direct
- Investing Cash Flow
- Financing Cash Flow

In [113]:
kept_values = [
    'Total Revenue',
    'Net Income',
    'Total Assets',
    'Total Liabilities Net Minority Interest',
    'Cash Flowsfromusedin Operating Activities Direct',
    'Investing Cash Flow',
    'Financing Cash Flow'
]

cleaned_data = scrapped_data[scrapped_data['account'].isin(kept_values)]

In [114]:
cleaned_data['account'] = cleaned_data['account'].replace(
    'Cash Flowsfromusedin Operating Activities Direct', 
    'Operating Cash Flow'
)

cleaned_data['account'] = cleaned_data['account'].replace(
    'Total Liabilities Net Minority Interest', 
    'Total Liabilities'
)

### Table Transformation

In [115]:
#  Melt table to long format
df_long = pd.melt(cleaned_data, id_vars=['symbol', 'account', 'type'], var_name='Year', value_name='Value')

# Pivot tabel to get desired format
df_pivot = df_long.pivot(index=['symbol', 'Year'], columns='account', values='Value').reset_index()

# Change column names
df_pivot.columns.name = None  # Delete col name
df_pivot = df_pivot.rename_axis(None, axis=1)

In [116]:
# Convert all numerical columns to float64
for col in ['Financing Cash Flow', 'Investing Cash Flow', 'Operating Cash Flow', 'Net Income', 'Total Revenue', 'Total Assets', 'Total Liabilities']:
    df_pivot[col] = pd.to_numeric(df_pivot[col], errors='coerce')

df_pivot['Year'] = df_pivot['Year'].astype('int64')

### Add additional columns

In [117]:
# Add Liabilities to Assets % Column
df_pivot['Liabilities to Assets %'] = (df_pivot['Total Liabilities'] / df_pivot['Total Assets']) * 100

# Add Net Margin % Column
df_pivot['Net Margin %'] = (df_pivot['Net Income'] / df_pivot['Total Revenue']) * 100

In [118]:
# Order Columns for better readibility
ordered_columns = ['symbol', 'Year', 'Financing Cash Flow', 'Investing Cash Flow', 'Operating Cash Flow', 'Total Revenue', 'Net Income', 'Net Margin %', 'Total Assets', 'Total Liabilities', 'Liabilities to Assets %']
df_financial = df_pivot[ordered_columns]

In [119]:
df_financial

,symbol,Year,Financing Cash Flow,Investing Cash Flow,Operating Cash Flow,Total Revenue,Net Income,Net Margin %,Total Assets,Total Liabilities,Liabilities to Assets %
0,ACES,2020,-4.050379e+11,-1.659800e+11,1.538615e+12,7.412767e+12,7.331955e+11,9.890982,7.216725e+12,1.873127e+12,25.955363
1,ACES,2021,-8.978434e+11,-8.337988e+10,1.305426e+12,6.543363e+12,6.907705e+11,10.556812,7.171138e+12,1.592158e+12,22.202307
2,ACES,2022,-9.394425e+11,-1.023613e+11,6.188384e+11,6.762803e+12,6.643429e+11,9.823483,7.249255e+12,1.315266e+12,18.143465
3,ACES,2023,-1.041742e+12,-1.306747e+11,1.356430e+12,7.611866e+12,7.635075e+11,10.030490,7.753269e+12,1.566872e+12,20.209172
4,ADRO,2020,-7.779670e+08,-3.616570e+08,7.387530e+08,2.534842e+09,1.469270e+08,5.796298,6.381566e+09,2.429852e+09,38.076109
...,...,...,...,...,...,...,...,...,...,...,...
175,UNTR,2023,-1.223004e+13,-3.343977e+13,2.634685e+13,1.285833e+14,2.061178e+13,16.029905,1.540282e+14,6.999268e+13,45.441460
176,UNVR,2020,-7.458509e+12,-6.902160e+11,8.363993e+12,4.297247e+13,7.163536e+12,16.670057,2.053463e+13,1.559726e+13,75.955897
177,UNVR,2021,-7.739754e+12,-6.812550e+11,7.902091e+12,3.954596e+13,5.758148e+12,14.560648,1.906853e+13,1.474726e+13,77.338219
178,UNVR,2022,-7.357788e+12,-5.260630e+11,8.061314e+12,4.121888e+13,5.364761e+12,13.015300,1.831811e+13,1.432086e+13,78.178671


In [120]:
# Folder tempat file CSV disimpan
folder_path = 'datasets/lq45_stock_performance'

# List untuk menyimpan DataFrame sementara
dataframes = []

# Membaca semua file CSV dalam folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        symbol = filename.split('.')[0]  # Mendapatkan symbol dari nama file
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        df['symbol'] = symbol  # Menambahkan kolom symbol
        dataframes.append(df)

# Menggabungkan semua DataFrame menjadi satu
combined_df = pd.concat(dataframes, ignore_index=True)

# Mengubah kolom 'Date' ke tipe datetime
combined_df['Date'] = pd.to_datetime(combined_df['Date'])

# Membatasi data dari tanggal 2004-01-01
filtered_df = combined_df[combined_df['Date'] >= '2004-01-01']

# Order Columns for better readibility
ordered_columns = ['symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
fact_stock_prices = filtered_df[ordered_columns]


In [139]:
fact_stock_prices

,id,symbol,Date,Open,High,Low,Close,Adj Close,Volume
0,0,ACES,2007-11-06,98.0,102.0,89.0,98.0,75.376373,1.274430e+09
1,1,ACES,2007-11-07,98.0,103.0,97.0,101.0,77.683800,3.493300e+08
2,2,ACES,2007-11-08,99.0,100.0,96.0,99.0,76.145500,6.627000e+07
3,3,ACES,2007-11-09,95.0,97.0,95.0,95.0,73.068932,4.007500e+07
4,4,ACES,2007-11-12,90.0,95.0,89.0,90.0,69.223190,1.132850e+08
...,...,...,...,...,...,...,...,...,...
177045,177045,UNVR,2024-05-17,2700.0,2750.0,2700.0,2710.0,2710.000000,9.948800e+06
177046,177046,UNVR,2024-05-20,2710.0,2750.0,2680.0,2700.0,2700.000000,6.973900e+06
177047,177047,UNVR,2024-05-21,2710.0,2910.0,2700.0,2910.0,2910.000000,4.462410e+07
177048,177048,UNVR,2024-05-22,2910.0,3090.0,2870.0,3060.0,3060.000000,5.045250e+07


### Transforming Data to Star Schema

#### Financial Fact

In [122]:
df_financial

,symbol,Year,Financing Cash Flow,Investing Cash Flow,Operating Cash Flow,Total Revenue,Net Income,Net Margin %,Total Assets,Total Liabilities,Liabilities to Assets %
0,ACES,2020,-4.050379e+11,-1.659800e+11,1.538615e+12,7.412767e+12,7.331955e+11,9.890982,7.216725e+12,1.873127e+12,25.955363
1,ACES,2021,-8.978434e+11,-8.337988e+10,1.305426e+12,6.543363e+12,6.907705e+11,10.556812,7.171138e+12,1.592158e+12,22.202307
2,ACES,2022,-9.394425e+11,-1.023613e+11,6.188384e+11,6.762803e+12,6.643429e+11,9.823483,7.249255e+12,1.315266e+12,18.143465
3,ACES,2023,-1.041742e+12,-1.306747e+11,1.356430e+12,7.611866e+12,7.635075e+11,10.030490,7.753269e+12,1.566872e+12,20.209172
4,ADRO,2020,-7.779670e+08,-3.616570e+08,7.387530e+08,2.534842e+09,1.469270e+08,5.796298,6.381566e+09,2.429852e+09,38.076109
...,...,...,...,...,...,...,...,...,...,...,...
175,UNTR,2023,-1.223004e+13,-3.343977e+13,2.634685e+13,1.285833e+14,2.061178e+13,16.029905,1.540282e+14,6.999268e+13,45.441460
176,UNVR,2020,-7.458509e+12,-6.902160e+11,8.363993e+12,4.297247e+13,7.163536e+12,16.670057,2.053463e+13,1.559726e+13,75.955897
177,UNVR,2021,-7.739754e+12,-6.812550e+11,7.902091e+12,3.954596e+13,5.758148e+12,14.560648,1.906853e+13,1.474726e+13,77.338219
178,UNVR,2022,-7.357788e+12,-5.260630e+11,8.061314e+12,4.121888e+13,5.364761e+12,13.015300,1.831811e+13,1.432086e+13,78.178671


In [123]:
# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Data for cash_flow_dim
cash_flow_dim = df_financial[['Financing Cash Flow', 'Investing Cash Flow' ,'Operating Cash Flow']]
cash_flow_dim.columns = ['financing_cash_flow', 'investing_cash_flow', 'operating_cash_flow']
cash_flow_dim['cash_flow_id'] = cash_flow_dim.index
ordered_columns = ['cash_flow_id', 'financing_cash_flow', 'investing_cash_flow', 'operating_cash_flow']
cash_flow_dim = cash_flow_dim[ordered_columns]

# Data for income_statement_dim
income_statement_dim = df_financial[['Net Income', 'Total Revenue', 'Net Margin %']]
income_statement_dim.columns = ['net_income', 'total_revenue', 'net_margin_percent']
income_statement_dim['income_statement_id'] = cash_flow_dim.index
ordered_columns = ['income_statement_id', 'net_income', 'total_revenue', 'net_margin_percent']
income_statement_dim = income_statement_dim[ordered_columns]

# Data untuk balance_sheet_dim
balance_sheet_dim = df_financial[['Total Assets', 'Total Liabilities', 'Liabilities to Assets %']]
balance_sheet_dim.columns = ['total_assets', 'total_liabilities', 'liabilities_to_assets_percent']
balance_sheet_dim['balance_sheet_id'] = cash_flow_dim.index
ordered_columns = ['balance_sheet_id', 'total_assets', 'total_liabilities', 'liabilities_to_assets_percent']
balance_sheet_dim = balance_sheet_dim[ordered_columns]

# Tabel fact_financials
fact_financials = df_financial[['symbol', 'Year']]
fact_financials['cash_flow_id'] = cash_flow_dim['cash_flow_id']
fact_financials['income_statement_id'] = income_statement_dim['income_statement_id']
fact_financials['balance_sheet_id'] = balance_sheet_dim['balance_sheet_id']
fact_financials['id'] = fact_financials.index
ordered_columns = ['id','symbol','Year','cash_flow_id','income_statement_id','balance_sheet_id']
fact_financials = fact_financials[ordered_columns]

Key Stats and company dimensions

In [124]:
def get_market_cap(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        market_cap = ticker.info.get('marketCap')
        return market_cap
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None
    
def get_shares_outstanding(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        shares_outstanding = ticker.info.get('sharesOutstanding')
        return shares_outstanding
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None
    
def get_dividend_yield(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        dividend_yield = ticker.info.get('dividendYield')
        return dividend_yield
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def get_eps(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        eps = ticker.info.get('trailingEps')
        return eps
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None
    
def get_pe_ratio(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        pe_ratio = ticker.info.get('trailingPE')
        return pe_ratio
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None
    
def get_company_name(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        company_name = ticker.info.get('longName')
        return company_name
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def get_city(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        city = ticker.info.get('city')
        return city
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def get_sector(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        sector = ticker.info.get('sector')
        return sector
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def get_industry(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        industry = ticker.info.get('industry')
        return industry
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None
    
def get_summary(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.JK")
        summary = ticker.info.get('longBusinessSummary')
        return summary
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

In [125]:
key_stats_dim = df_financial[['symbol']]
key_stats_dim = key_stats_dim.drop_duplicates(subset='symbol').reset_index(drop=True)

company_dim = df_financial[['symbol']]
company_dim = company_dim.drop_duplicates(subset='symbol').reset_index(drop=True)

In [126]:
# Key Stats Dimension (Current)
key_stats_dim['market_cap'] =  key_stats_dim['symbol'].apply(get_market_cap)
key_stats_dim['shares_outstanding'] =  key_stats_dim['symbol'].apply(get_shares_outstanding)
key_stats_dim['dividend_yield'] =  key_stats_dim['symbol'].apply(get_dividend_yield)
key_stats_dim['eps'] =  key_stats_dim['symbol'].apply(get_eps)
key_stats_dim['pe_ratio'] =  key_stats_dim['symbol'].apply(get_pe_ratio)

In [127]:
## Company Dimension
company_dim['company_name'] =  company_dim['symbol'].apply(get_company_name)
company_dim['city'] =  company_dim['symbol'].apply(get_city)
company_dim['sector'] =  company_dim['symbol'].apply(get_sector)
company_dim['industry'] =  company_dim['symbol'].apply(get_industry)
company_dim['summary'] =  company_dim['symbol'].apply(get_summary)

#### Stock Price Fact

In [128]:
fact_stock_prices['id'] = fact_stock_prices.index
ordered_columns = ['id','symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
fact_stock_prices = fact_stock_prices[ordered_columns]

In [129]:
date_dim = fact_stock_prices[['Date']]
date_dim = date_dim.drop_duplicates(subset='Date').reset_index(drop=True)

In [130]:
# Date Dimension
date_dim['day'] = date_dim['Date'].dt.day
date_dim['month'] = date_dim['Date'].dt.month
date_dim['year'] = date_dim['Date'].dt.year

### Table Result

In [131]:
fact_financials

,id,symbol,Year,cash_flow_id,income_statement_id,balance_sheet_id
0,0,ACES,2020,0,0,0
1,1,ACES,2021,1,1,1
2,2,ACES,2022,2,2,2
3,3,ACES,2023,3,3,3
4,4,ADRO,2020,4,4,4
...,...,...,...,...,...,...
175,175,UNTR,2023,175,175,175
176,176,UNVR,2020,176,176,176
177,177,UNVR,2021,177,177,177
178,178,UNVR,2022,178,178,178


In [132]:
fact_stock_prices

,id,symbol,Date,Open,High,Low,Close,Adj Close,Volume
0,0,ACES,2007-11-06,98.0,102.0,89.0,98.0,75.376373,1.274430e+09
1,1,ACES,2007-11-07,98.0,103.0,97.0,101.0,77.683800,3.493300e+08
2,2,ACES,2007-11-08,99.0,100.0,96.0,99.0,76.145500,6.627000e+07
3,3,ACES,2007-11-09,95.0,97.0,95.0,95.0,73.068932,4.007500e+07
4,4,ACES,2007-11-12,90.0,95.0,89.0,90.0,69.223190,1.132850e+08
...,...,...,...,...,...,...,...,...,...
177045,177045,UNVR,2024-05-17,2700.0,2750.0,2700.0,2710.0,2710.000000,9.948800e+06
177046,177046,UNVR,2024-05-20,2710.0,2750.0,2680.0,2700.0,2700.000000,6.973900e+06
177047,177047,UNVR,2024-05-21,2710.0,2910.0,2700.0,2910.0,2910.000000,4.462410e+07
177048,177048,UNVR,2024-05-22,2910.0,3090.0,2870.0,3060.0,3060.000000,5.045250e+07


In [133]:
cash_flow_dim

,cash_flow_id,financing_cash_flow,investing_cash_flow,operating_cash_flow
0,0,-4.050379e+11,-1.659800e+11,1.538615e+12
1,1,-8.978434e+11,-8.337988e+10,1.305426e+12
2,2,-9.394425e+11,-1.023613e+11,6.188384e+11
3,3,-1.041742e+12,-1.306747e+11,1.356430e+12
4,4,-7.779670e+08,-3.616570e+08,7.387530e+08
...,...,...,...,...
175,175,-1.223004e+13,-3.343977e+13,2.634685e+13
176,176,-7.458509e+12,-6.902160e+11,8.363993e+12
177,177,-7.739754e+12,-6.812550e+11,7.902091e+12
178,178,-7.357788e+12,-5.260630e+11,8.061314e+12


In [134]:
income_statement_dim

,income_statement_id,net_income,total_revenue,net_margin_percent
0,0,7.331955e+11,7.412767e+12,9.890982
1,1,6.907705e+11,6.543363e+12,10.556812
2,2,6.643429e+11,6.762803e+12,9.823483
3,3,7.635075e+11,7.611866e+12,10.030490
4,4,1.469270e+08,2.534842e+09,5.796298
...,...,...,...,...
175,175,2.061178e+13,1.285833e+14,16.029905
176,176,7.163536e+12,4.297247e+13,16.670057
177,177,5.758148e+12,3.954596e+13,14.560648
178,178,5.364761e+12,4.121888e+13,13.015300


In [135]:
balance_sheet_dim

,balance_sheet_id,total_assets,total_liabilities,liabilities_to_assets_percent
0,0,7.216725e+12,1.873127e+12,25.955363
1,1,7.171138e+12,1.592158e+12,22.202307
2,2,7.249255e+12,1.315266e+12,18.143465
3,3,7.753269e+12,1.566872e+12,20.209172
4,4,6.381566e+09,2.429852e+09,38.076109
...,...,...,...,...
175,175,1.540282e+14,6.999268e+13,45.441460
176,176,2.053463e+13,1.559726e+13,75.955897
177,177,1.906853e+13,1.474726e+13,77.338219
178,178,1.831811e+13,1.432086e+13,78.178671


In [136]:
key_stats_dim

,symbol,market_cap,shares_outstanding,dividend_yield,eps,pe_ratio
0,ACES,13953126105088,17120400384,0.0381,47.06,17.318317
1,ADRO,82452802437120,30881200128,0.1394,766.65,3.482684
2,AKRA,31776892256256,19737200640,0.0791,140.30,11.475409
3,AMRT,116268598296576,41524498432,0.0102,87.26,32.088013
4,ANTM,36166355124224,24030799872,0.0857,69.69,21.595636
5,ARTO,32562070159360,13856199680,NaN,5.50,427.272740
6,ASII,178127829467136,40483598336,0.1141,797.82,5.515028
7,BBCA,1134130003705856,123275001856,0.0290,395.25,23.213156
8,BBNI,167986673483776,578684032,0.0601,568.97,7.926605
9,BBRI,669512051458048,150790995968,0.0701,398.19,11.125342


In [137]:
company_dim

,symbol,company_name,city,sector,industry,summary
0,ACES,PT Ace Hardware Indonesia Tbk,Jakarta,Consumer Cyclical,Specialty Retail,"PT Ace Hardware Indonesia Tbk, together with i..."
1,ADRO,PT Adaro Energy Indonesia Tbk,Jakarta,Energy,Thermal Coal,"PT Adaro Energy Indonesia Tbk, together with i..."
2,AKRA,PT AKR Corporindo Tbk,Jakarta,Energy,Oil & Gas Refining & Marketing,"PT AKR Corporindo Tbk, a logistics and supply ..."
3,AMRT,PT Sumber Alfaria Trijaya Tbk,Tangerang,Consumer Defensive,Grocery Stores,PT Sumber Alfaria Trijaya Tbk engages in the r...
4,ANTM,PT Aneka Tambang Tbk,Jakarta,Basic Materials,Gold,PT Aneka Tambang Tbk operates as a diversified...
5,ARTO,PT Bank Jago Tbk,Jakarta,Financial Services,Banks - Regional,PT Bank Jago Tbk provides various banking prod...
6,ASII,PT Astra International Tbk,Jakarta,Industrials,Conglomerates,"PT Astra International Tbk, together with its ..."
7,BBCA,PT Bank Central Asia Tbk,Jakarta,Financial Services,Banks - Regional,"PT Bank Central Asia Tbk, together with its su..."
8,BBNI,PT Bank Negara Indonesia (Persero) Tbk,Jakarta Pusat,Financial Services,Banks - Regional,"PT Bank Negara Indonesia (Persero) Tbk, togeth..."
9,BBRI,PT Bank Rakyat Indonesia (Persero) Tbk,Jakarta,Financial Services,Banks - Regional,PT Bank Rakyat Indonesia (Persero) Tbk provide...


In [138]:
date_dim

,Date,day,month,year
0,2007-11-06,6,11,2007
1,2007-11-07,7,11,2007
2,2007-11-08,8,11,2007
3,2007-11-09,9,11,2007
4,2007-11-12,12,11,2007
...,...,...,...,...
5045,2005-09-22,22,9,2005
5046,2005-09-23,23,9,2005
5047,2005-09-26,26,9,2005
5048,2005-09-27,27,9,2005
